# Setup

In [39]:
import os
import os.path
import pandas as pd
import importlib
import json
import numpy as np

In [40]:
path = '/cs/labs/michall/roeizucker/phewas_run/runs1/0:9'
COVARIATE_JSON_PATH = path + "/ukbb_covariate_columns.json"
GWAS_PATH = path + "/GWAS_delete_me"
PHEN_PATH = path + "/ukbb_dataset.csv"
PLINK_PATH =  "/sci/home/nadavb/third_party/plink2"
# PHEN_PER_DIR = 12
PHEN_PER_DIR = 10

PHEN_START = 3
MAX_ARRAYS_BEFRE_SUBMITTING = 0
MAX_JOBS_BEFORE_SUBMITTING_NEW_ARRAY = 200


In [41]:
!{PLINK_PATH} --version

PLINK v2.00a2.3LM 64-bit Intel (24 Jan 2020)


In [42]:
line = ""
with open(PHEN_PATH) as phen_file:
    line = phen_file.readline()
phenotypes = line.split(",")[PHEN_START - 1:PHEN_START + PHEN_PER_DIR - 1]
print(phenotypes)

['Phecode_8.0', 'Phecode_8.0_F', 'Phecode_8.0_M', 'Phecode_8.5', 'Phecode_8.5_F', 'Phecode_8.5_M', 'Phecode_8.51', 'Phecode_8.51_F', 'Phecode_8.51_M', 'Phecode_8.52']


In [43]:
!mkdir {GWAS_PATH}
!mkdir {GWAS_PATH}/results
!mkdir {GWAS_PATH}/phenotypes
!touch {GWAS_PATH}/pan_cancer.log
!touch {GWAS_PATH}/pan_cancer.txt
!touch {GWAS_PATH}/covariates.txt


mkdir: cannot create directory ‘/cs/labs/michall/roeizucker/phewas_run/runs1/0:9/GWAS_delete_me’: File exists
mkdir: cannot create directory ‘/cs/labs/michall/roeizucker/phewas_run/runs1/0:9/GWAS_delete_me/results’: File exists
mkdir: cannot create directory ‘/cs/labs/michall/roeizucker/phewas_run/runs1/0:9/GWAS_delete_me/phenotypes’: File exists


In [6]:
# !cd /cs/labs/michall/roeizucker/10krun/runs/0:11/GWAS_delete_me
# for i in range(len(phenotypes)):
#     var = f"cat {PHEN_PATH} | awk -F , '{{print $2\",\"$2\",\"$'{i + PHEN_START}'}}' | grep  '\.' > {GWAS_PATH}/phenotypes/{phenotypes[i]}.csv"
# #     print(var)
#     !{var}

In [44]:
PHEN_END = 10
WORK_PATH = '/cs/labs/michall/roeizucker/phewas_run/runs1/'
PHEN_PER_DIR = 10
PHEN_START = 3

In [45]:
def create_files(GWAS_PATH,PHEN_PATH,phenotypes):
    !mkdir {GWAS_PATH}
    !mkdir {GWAS_PATH}/results
    !mkdir {GWAS_PATH}/phenotypes
    !cp /cs/labs/michall/roeizucker/10krun/runs/0:11/GWAS_delete_me/covariates.txt {GWAS_PATH}/covariates.txt
    !cd {GWAS_PATH}
    cancer_dataset = pd.read_csv(PHEN_PATH)
    cancer_phenotype_cols = phenotypes
    for cancer_phenotype_col in cancer_phenotype_cols:
        print(cancer_phenotype_col)
        cancer_file_name = cancer_phenotype_col.lower().replace(' ', '_').replace('-', '_') + '.txt'
        cancer_values = cancer_dataset[['eid', 'eid', cancer_phenotype_col]].dropna()
        cancer_values.to_csv(os.path.join(f"{GWAS_PATH}/phenotypes", cancer_file_name), \
            header = False, index = False, sep = '\t')


In [46]:
def handle_dir(path):
    COVARIATE_JSON_PATH = path + "/ukbb_covariate_columns.json"
    GWAS_PATH = path + "/GWAS"
    PHEN_PATH = path + "/ukbb_dataset.csv"
    line = ""
    with open(PHEN_PATH) as phen_file:
        line = phen_file.readline()
    phenotypes = line.split(",")[PHEN_START - 1:PHEN_START + PHEN_PER_DIR - 1]
    create_files(GWAS_PATH,PHEN_PATH,phenotypes)
#     print(path)

In [47]:
def create_script(path):
    COVARIATE_JSON_PATH = path + "/ukbb_covariate_columns.json"
    GWAS_PATH = path + "/GWAS"
    PHEN_PATH = path + "/ukbb_dataset.csv"
    line = ""
    with open(PHEN_PATH) as phen_file:
        line = phen_file.readline()
    phenotypes = line.split(",")[PHEN_START - 1:PHEN_START + PHEN_PER_DIR - 1]
    !rm -f {GWAS_PATH}/gwas_script.sh
    with open(f"{GWAS_PATH}/gwas_script.sh", 'a') as script_file:
        script_file.write("#!/bin/sh\n")
        for i in range(len(phenotypes)):
            for j in range(1,23):
        #         print(f"/cs/phd/nadavb/third_party/plink2 --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.fam --pheno {GWAS_PATH}/phenotypes/{phenotypes[i].lower()}.txt --covar {GWAS_PATH}/covariates.txt --out {GWAS_PATH}/results/{phenotypes[i]}_chr{j} --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000")
                script_file.write(f"{PLINK_PATH} --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.fam --pheno {GWAS_PATH}/phenotypes/{phenotypes[i].lower()}.txt --covar {GWAS_PATH}/covariates.txt --out {GWAS_PATH}/results/{phenotypes[i]}_chr{j} --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000\n")
    !chmod 744 {GWAS_PATH}/gwas_script.sh
#     print(path)

In [48]:
def create_slurm_script(path):
    COVARIATE_JSON_PATH = path + "/ukbb_covariate_columns.json"
    GWAS_PATH = path + "/GWAS"
    PHEN_PATH = path + "/ukbb_dataset.csv"
    line = ""
    with open(PHEN_PATH) as phen_file:
        line = phen_file.readline()
    phenotypes = line.split(",")[PHEN_START - 1:PHEN_START + PHEN_PER_DIR - 1]
    !rm -f {GWAS_PATH}/slurm_script.sh
    with open(f"{GWAS_PATH}/slurm_script.sh", 'a') as script_file:
        script_file.write("#!/bin/bash\n\nsmart_wait () {\nNUM=`squeue | grep roeizuck | wc -l`\nwhile [ $NUM -gt 1000 ]\ndo\n        NUM=`squeue | grep roeizuck | wc -l`\n        sleep 10\ndone\n}\n")
        for i in range(len(phenotypes)):
            for j in range(1,23):
                if os.path.exists(f"{GWAS_PATH}/results/{phenotypes[i]}_chr{j}.PHENO1.glm.logistic"):
                    continue
        #         print(f"/cs/phd/nadavb/third_party/plink2 --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.fam --pheno {GWAS_PATH}/phenotypes/{phenotypes[i].lower()}.txt --covar {GWAS_PATH}/covariates.txt --out {GWAS_PATH}/results/{phenotypes[i]}_chr{j} --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000")
                script_file.write(f"echo {GWAS_PATH}/results/{phenotypes[i]}_chr{j}\n")
                script_file.write("sbatch --mem=12g -c10 --time=3-0 --killable --requeue --wrap=\"")
                script_file.write(f"{PLINK_PATH} --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.fam --pheno {GWAS_PATH}/phenotypes/{phenotypes[i].lower()}.txt --covar {GWAS_PATH}/covariates.txt --out {GWAS_PATH}/results/{phenotypes[i]}_chr{j} --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000")
                script_file.write("\"\n")
                script_file.write("sleep 5\n")
                script_file.write("smart_wait\n")
    !chmod 744 {GWAS_PATH}/slurm_script.sh
#     print(path)

In [49]:
def create_slurm_array_script(path):
    COVARIATE_JSON_PATH = path + "/ukbb_covariate_columns.json"
    GWAS_PATH = path + "/GWAS"
    PHEN_PATH = path + "/ukbb_dataset.csv"
    line = ""
    with open(PHEN_PATH) as phen_file:
        line = phen_file.readline()
    phenotypes = line.split(",")[PHEN_START - 1:PHEN_START + PHEN_PER_DIR - 1]
    !rm -f {GWAS_PATH}/array_slurm_jobs.py
    with open(f"{GWAS_PATH}/array_slurm_jobs.py", 'a') as script_file:
        # import sys library (needed for accepted command line args)
        script_file.write("import sys\n")
        script_file.write("import os\n")
        # print arguments
#         print( sys.argv)
        command = "os.system(\""+GWAS_PATH+"/array_slurm_jobs_\"+str(sys.argv[1])+\".sh\")\n"
        script_file.write(command)
    !rm -f {GWAS_PATH}/array_slurm_jobs_runner.sh
    !rm -f {GWAS_PATH}/array_slurm_jobs_runner.py
    with open(f"{GWAS_PATH}/array_slurm_jobs_runner.sh", 'a') as runner:
        # import sys library (needed for accepted command line args)
        runner.write(f"python {GWAS_PATH}/array_slurm_jobs.py $SLURM_ARRAY_TASK_ID $SLURM_ARRAY_TASK_COUNT\n")
    !chmod 744 {GWAS_PATH}/array_slurm_jobs_runner.sh
    print(path)
    counter = 0
    for i in range(len(phenotypes)):
        for j in range(1,23):
            !rm -f {GWAS_PATH}/array_slurm_jobs_{counter}.sh
            if os.path.exists(f"{GWAS_PATH}/results/{phenotypes[i]}_chr{j}.PHENO1.glm.logistic"):
                if counter == MAX_JOBS_BEFORE_SUBMITTING_NEW_ARRAY:
                    with open(f"{GWAS_PATH}/array_slurm_jobs_{counter}.sh", 'a') as script_file:
                        script_file.write("#!/bin/bash\n")
                        phen_range = path.split("/")[-1].split(":")
                        first = int(phen_range[0]) + PHEN_PER_DIR
                        last  = int(phen_range[1]) + PHEN_PER_DIR
#                         script_file.write("sbatch --array=0-263 --mem=12g -c10 --time=3-0 --requeue --killable --wrap=\"/cs/labs/michall/roeizucker/10krun/runs/"+str(first)+":"+str(last)+"/GWAS/array_slurm_jobs_runner.sh\"\n")
#                         TODO: change 263 to lower number, 230?
                        script_file.write("sbatch --array=0-263 --mem=12g -c10 --time=3-0 --requeue --killable --wrap=\"" + WORK_PATH +str(first)+":"+str(last)+"/GWAS/array_slurm_jobs_runner.sh\"\n")
                    !chmod 744 {GWAS_PATH}/array_slurm_jobs_{counter}.sh
                counter+=1
                continue
    #         print(f"/cs/phd/nadavb/third_party/plink2 --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.fam --pheno {GWAS_PATH}/phenotypes/{phenotypes[i].lower()}.txt --covar {GWAS_PATH}/covariates.txt --out {GWAS_PATH}/results/{phenotypes[i]}_chr{j} --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000")
            with open(f"{GWAS_PATH}/array_slurm_jobs_{counter}.sh", 'a') as script_file:
                if counter == MAX_JOBS_BEFORE_SUBMITTING_NEW_ARRAY:
                    script_file.write('''#!/bin/bash

smart_wait () {
NUM=`squeue -u roeizucker | grep '\[' | wc -l`
while [ $NUM -gt 1 ]
do
        NUM=`squeue | grep roeizuck | grep '\[' | wc -l`
        sleep 10
done
}
''')
                else:
                    script_file.write("#!/bin/bash\n")
                if counter == MAX_JOBS_BEFORE_SUBMITTING_NEW_ARRAY:
                    phen_range = path.split("/")[-1].split(":")
                    first = int(phen_range[0]) + PHEN_PER_DIR
                    last  = int(phen_range[1]) + PHEN_PER_DIR
                    script_file.write("smart_wait\n")
                    script_file.write("sbatch --array=0-263 --mem=12g -c10 --time=3-0 --requeue --killable --wrap=\"" + WORK_PATH +str(first)+":"+str(last)+"/GWAS/array_slurm_jobs_runner.sh\"\n")
                script_file.write(f"{PLINK_PATH} --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.fam --pheno {GWAS_PATH}/phenotypes/{phenotypes[i].lower()}.txt --covar {GWAS_PATH}/covariates.txt --out {GWAS_PATH}/results/{phenotypes[i]}_chr{j} --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000\n")
                !chmod 744 {GWAS_PATH}/array_slurm_jobs_{counter}.sh
            counter+=1
    print("done")

### create script for full genom for a single phenotype

In [16]:
sex_suffix = ["","_F","_M"]
phen = "I10"
GWAS_PATH = "/cs/labs/michall/roeizucker/10krun/runs/48:59/GWAS"
counter = 0
for suf in sex_suffix:
    for i in range(1,23):
        with open(f"{GWAS_PATH}/full_genom_scripts/array_slurm_jobs_{counter}.sh", 'a') as script_file:
            script_file.write(f"{PLINK_PATH} --bgen /cs/biobank-genetics/EGAD00010001474/ukb_imp_chr{i}_v3.bgen --sample /cs/biobank-genetics/EGAD00010001474/ukb26664_imp_chr{i}_v3.sample --pheno /cs/labs/michall/roeizucker/10krun/runs/48:59/GWAS/phenotypes/{phen}{suf}.csv --covar /cs/labs/michall/roeizucker/10krun/runs/240:251/GWAS/covariates.txt --out /cs/labs/michall/roeizucker/10krun/runs/48:59/GWAS/results_full/I{phen}{suf}_chr{i} --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000")
        counter+=1
print(counter)
# sbatch --array=0-65 --mem=12g -c10 --time=3-0 --requeue --killable --wrap="/cs/labs/michall/roeizucker/10krun/runs/48:59/GWAS/full_genom_scripts/array_slurm_jobs_runner.sh"

66


## array based slurm 

In [63]:
import os
import pandas as pd
cont = False
# input()
for filename in os.listdir(WORK_PATH):
#     break
    f = os.path.join(WORK_PATH, filename)
    # checking if it is a file
    print(f)
    if f == '/cs/labs/michall/roeizucker/phewas_run/runs1/gwas' or \
    f == "/cs/labs/michall/roeizucker/phewas_run/runs1/4620:4629" or \
    f == "/cs/labs/michall/roeizucker/phewas_run/runs1/priority_GWAS":
        continue    
    if not os.path.isfile(f):
#         print(f)
#         input()
        if f == "/cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439":
            handle_dir(f)
        if os.path.isdir(f+ "/GWAS") or f == "/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results":
            continue
#         !sed 's/phd/usr/g' {f}/GWAS/*.sh -i
        handle_dir(f)
#         create_slurm_array_script(f)
#         create_script(f)
#         create_slurm_script(f)

/cs/labs/michall/roeizucker/phewas_run/runs1/4940:4949
/cs/labs/michall/roeizucker/phewas_run/runs1/1500:1509
/cs/labs/michall/roeizucker/phewas_run/runs1/850:859
/cs/labs/michall/roeizucker/phewas_run/runs1/730:739
/cs/labs/michall/roeizucker/phewas_run/runs1/210:219
/cs/labs/michall/roeizucker/phewas_run/runs1/4200:4209
/cs/labs/michall/roeizucker/phewas_run/runs1/2770:2779
/cs/labs/michall/roeizucker/phewas_run/runs1/3060:3069
/cs/labs/michall/roeizucker/phewas_run/runs1/4420:4429
/cs/labs/michall/roeizucker/phewas_run/runs1/50:59
/cs/labs/michall/roeizucker/phewas_run/runs1/4390:4399
/cs/labs/michall/roeizucker/phewas_run/runs1/1860:1869
/cs/labs/michall/roeizucker/phewas_run/runs1/3640:3649
/cs/labs/michall/roeizucker/phewas_run/runs1/2150:2159
/cs/labs/michall/roeizucker/phewas_run/runs1/1320:1329
/cs/labs/michall/roeizucker/phewas_run/runs1/1490:1499
/cs/labs/michall/roeizucker/phewas_run/runs1/1420:1429
/cs/labs/michall/roeizucker/phewas_run/runs1/1390:1399
/cs/labs/michall/roe

mkdir: cannot create directory ‘/cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439/GWAS’: File exists
mkdir: cannot create directory ‘/cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439/GWAS/results’: File exists
mkdir: cannot create directory ‘/cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439/GWAS/phenotypes’: File exists
Phecode_508.0
Phecode_508.0_F
Phecode_508.0_M
Phecode_509.1
Phecode_509.1_F
Phecode_509.1_M
Phecode_509.2
Phecode_509.2_F
Phecode_509.2_M
Phecode_509.3
/cs/labs/michall/roeizucker/phewas_run/runs1/1640:1649
/cs/labs/michall/roeizucker/phewas_run/runs1/3860:3869
/cs/labs/michall/roeizucker/phewas_run/runs1/2870:2879
/cs/labs/michall/roeizucker/phewas_run/runs1/1140:1149
/cs/labs/michall/roeizucker/phewas_run/runs1/2480:2489
/cs/labs/michall/roeizucker/phewas_run/runs1/3420:3429
/cs/labs/michall/roeizucker/phewas_run/runs1/3390:3399
/cs/labs/michall/roeizucker/phewas_run/runs1/2330:2339
/cs/labs/michall/roeizucker/phewas_run/runs1/4640:4649
/cs/labs/michall

# priority based array slurm script

## discription:
we will create a file (data file) that represents all jobs that need to run.
there will be a master script and a worker script and a mediator script. the master script will activate the mediator script in batches of 250, and with a numeric parameter it recieved.
the mediator script will get the correct parameters from the data file (adding the current batch num times the number of tasks in eatch butch, added to the current task number).
the 200'th task will start the next master.

In [50]:
DISCRIPTION_FILE = "/cs/labs/michall/roeizucker/phewas_run/discription8_no_duplicates.csv"
priority_gwas_dir = WORK_PATH + "priority_GWAS/"
data_file_path = priority_gwas_dir + "data_file_2.csv"
mediator_script_path = priority_gwas_dir + "mediator.py"
master_script_path = priority_gwas_dir + "master.sh"
df = pd.read_csv(DISCRIPTION_FILE)
filtered_sorted_df = df[(df['num_relevant_genes_0.1'] > 0) & (df["source"] == "phewas")]
BATCH_LENGTH = 200
# WORK_PATH
# script_file.write(f"{PLINK_PATH} --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.fam --pheno {GWAS_PATH}/phenotypes/{phenotypes[i].lower()}.txt --covar {GWAS_PATH}/covariates.txt --out {GWAS_PATH}/results/{phenotypes[i]}_chr{j} --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000\n")

In [22]:
!mkdir {priority_gwas_dir}

mkdir: cannot create directory ‘/cs/labs/michall/roeizucker/phewas_run/runs1/priority_GWAS/’: File exists


In [23]:
data_file_path

'/cs/labs/michall/roeizucker/phewas_run/runs1/priority_GWAS/data_file_2.csv'

### create data file:

In [64]:
# An array is created with all the values needed for running GWAS.
# The array will be sorted in batches for better file accessing
values = []
last_counter = 0
cur_phen_counter = 0
counter = 0
for index, row in filtered_sorted_df.iterrows():
    base_path =  row['dataset_path'].replace("ukbb_dataset.csv","") + "GWAS/"
#     {GWAS_PATH}/results/{phenotypes[i]}_chr{j}
#     print("phenotype path", base_path+ "phenotypes/"+ row['name'].lower() + ".txt")
    for j in range(1,23):
        if os.path.exists(base_path + f"results/{row['name'].lower()}_chr{j}.PHENO1.glm.logistic"):
            continue
        values.append([counter,base_path+ "phenotypes/"+ row['name'].lower() + ".txt",
                       base_path + f"results/{row['name'].lower()}_chr{j}",
                       f"/cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}"
                       ,base_path +"covariates.txt",j])
        counter+=1
    cur_phen_counter+=1
    if cur_phen_counter%BATCH_LENGTH == 0:
        values[last_counter:counter] = sorted(values[last_counter:counter],key = lambda x:x[-1])
        last_counter = counter
values[last_counter:counter] = sorted(values[last_counter:counter],key = lambda x:x[-1])
        #         print("result_path", base_path + f"results/{row['name'].lower()}_chr{j}")

# filtered_sorted_df

In [65]:
df = pd.DataFrame(values, columns=["unsorted_counter","phenotype_path","output_path","partial_chromosome_file_path", "covariates","chr"])


In [66]:
list(df["phenotype_path"].unique())

['/cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439/GWAS/phenotypes/phecode_509.1.txt',
 '/cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439/GWAS/phenotypes/phecode_509.2_m.txt',
 '/cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439/GWAS/phenotypes/phecode_509.2.txt',
 '/cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439/GWAS/phenotypes/phecode_509.1_m.txt',
 '/cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439/GWAS/phenotypes/phecode_509.2_f.txt',
 '/cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439/GWAS/phenotypes/phecode_508.0.txt',
 '/cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439/GWAS/phenotypes/phecode_509.1_f.txt',
 '/cs/labs/michall/roeizucker/phewas_run/runs1/2330:2339/GWAS/phenotypes/phecode_476.0.txt']

In [67]:
df.to_csv(data_file_path)

In [68]:
# script_file.write(f"{PLINK_PATH} --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.fam --pheno {GWAS_PATH}/phenotypes/{phenotypes[i].lower()}.txt --covar {GWAS_PATH}/covariates.txt --out {GWAS_PATH}/results/{phenotypes[i]}_chr{j} --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000\n")
val = df.loc[3]["unsorted_counter"]

print(f"{PLINK_PATH} --bed {df.loc[3].partial_chromosome_file_path}.bed --bim {df.loc[3].partial_chromosome_file_path}.bim --fam {df.loc[3].partial_chromosome_file_path}.fam --pheno {df.loc[3].phenotype_path} --covar {df.loc[3].covariates} --out {df.loc[3].output_path} --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000\n")
df

/sci/home/nadavb/third_party/plink2 --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch1.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch1.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch1.fam --pheno /cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439/GWAS/phenotypes/phecode_509.1_m.txt --covar /cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439/GWAS/covariates.txt --out /cs/labs/michall/roeizucker/phewas_run/runs1/2430:2439/GWAS/results/phecode_509.1_m_chr1 --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000



,unsorted_counter,phenotype_path,output_path,partial_chromosome_file_path,covariates,chr
0,1,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/plink_results/redu...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,1
1,23,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/plink_results/redu...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,1
2,45,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/plink_results/redu...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,1
3,67,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/plink_results/redu...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,1
4,89,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/plink_results/redu...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,1
...,...,...,...,...,...,...
150,66,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/plink_results/redu...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,22
151,88,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/plink_results/redu...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,22
152,110,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/plink_results/redu...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,22
153,132,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,/cs/labs/michall/roeizucker/plink_results/redu...,/cs/labs/michall/roeizucker/phewas_run/runs1/2...,22


### create python mediator script

In [69]:

with open(mediator_script_path, "w") as mediator_file:
    mediator_file.write(f'''import pandas as pd
import sys
import os
import os.path
butch_num = int(sys.argv[1]) 
curr_task = int(sys.argv[2])
BATCH_LENGTH = 250
SKIP_POINT = 200
PLINK_PATH = "{PLINK_PATH}"

data_file_path = "{data_file_path}"
df = pd.read_csv(data_file_path)
print(df.loc[butch_num * BATCH_LENGTH + curr_task])
val = df.loc[butch_num * BATCH_LENGTH + curr_task]
location = butch_num * BATCH_LENGTH + curr_task
if curr_task == SKIP_POINT and location < 30500:
    new_batch  = butch_num + 1
    if not os.path.isfile("/cs/labs/michall/roeizucker/phewas_run/runs1/priority_GWAS/_" + str(new_batch) + "_flag"):
        os.system("sbatch --array=0-"+str(BATCH_LENGTH - 1)+" --mem=12g -c10 --time=3-0 --killable --requeue --wrap=\\"/cs/labs/michall/roeizucker/phewas_run/runs1/priority_GWAS/master.sh "+str(new_batch)+"\\"")
if location < 30500:
    #print(PLINK_PATH + " --bed " + df.loc[location].partial_chromosome_file_path + ".bed --bim " + df.loc[location].partial_chromosome_file_path + ".bim --fam "+df.loc[location].partial_chromosome_file_path + ".fam --pheno " + df.loc[location].phenotype_path + " --covar " + df.loc[location].covariates + "  --out  " + df.loc[location].output_path + " --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000")
    os.system(PLINK_PATH + " --bed " + df.loc[location].partial_chromosome_file_path + ".bed --bim " + df.loc[location].partial_chromosome_file_path + ".bim --fam "+df.loc[location].partial_chromosome_file_path + ".fam --pheno " + df.loc[location].phenotype_path + " --covar " + df.loc[location].covariates + "  --out  " + df.loc[location].output_path + " --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000")
''')

    




### create master script

In [70]:
with open(master_script_path, "w") as master_file:
    master_file.write(f'''
FILE = {priority_gwas_dir}_$1_flag
if ! test "$FILE" 
then 
    touch {priority_gwas_dir}_$1_flag
fi
python {mediator_script_path} $1 $SLURM_ARRAY_TASK_ID
''')
!chmod 755 {master_script_path}

In [71]:
master_script_path

'/cs/labs/michall/roeizucker/phewas_run/runs1/priority_GWAS/master.sh'

### remove old flag files

In [31]:
!rm {priority_gwas_dir}_*_flag -f

In [72]:
# how to run
# sbatch --array=0-263 --mem=12g -c10 --time=3-0 --requeue --killable --wrap="/cs/labs/michall/roeizucker/phewas_run/runs1/priority_GWAS/master.sh 0"

# combining the results

In [10]:
gwas_results_dir = os.path.join(WORK_PATH, "GWAS_results")
!mkdir {gwas_results_dir}

mkdir: cannot create directory ‘/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results’: File exists


In [12]:
print(WORK_PATH)
for filename in os.listdir(WORK_PATH):
    f = os.path.join(WORK_PATH, filename)
#     print(f)
    if os.path.isfile(f):
        continue
    print(f)
    f = os.path.join(f, "GWAS/results")
#     print(os.listdir(f))
    print(f)
    !cp {f}/*.logistic {gwas_results_dir}

/cs/labs/michall/roeizucker/phewas_run/runs1/
/cs/labs/michall/roeizucker/phewas_run/runs1/4940:4949
/cs/labs/michall/roeizucker/phewas_run/runs1/4940:4949/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/1500:1509
/cs/labs/michall/roeizucker/phewas_run/runs1/1500:1509/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/850:859
/cs/labs/michall/roeizucker/phewas_run/runs1/850:859/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/730:739
/cs/labs/michall/roeizucker/phewas_run/runs1/730:739/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/210:219
/cs/labs/michall/roeizucker/phewas_run/runs1/210:219/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4200:4209
/cs/labs/michall/roeizucker/phewas_run/runs1/4200:4209/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/2770:2779
/cs/labs/michall/roeizucker/phewas_run/runs1/2770:2779/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/3060:3069
/cs/labs/michall/roeizucker/phewas_run/runs1/3060:

/cs/labs/michall/roeizucker/phewas_run/runs1/2920:2929
/cs/labs/michall/roeizucker/phewas_run/runs1/2920:2929/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4710:4719
/cs/labs/michall/roeizucker/phewas_run/runs1/4710:4719/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/5000:5009
/cs/labs/michall/roeizucker/phewas_run/runs1/5000:5009/GWAS/results
cp: No match.
/cs/labs/michall/roeizucker/phewas_run/runs1/3570:3579
/cs/labs/michall/roeizucker/phewas_run/runs1/3570:3579/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/2260:2269
/cs/labs/michall/roeizucker/phewas_run/runs1/2260:2269/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4680:4689
/cs/labs/michall/roeizucker/phewas_run/runs1/4680:4689/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4130:4139
/cs/labs/michall/roeizucker/phewas_run/runs1/4130:4139/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/2440:2449
/cs/labs/michall/roeizucker/phewas_run/runs1/2440:2449/GWAS/results
/c

/cs/labs/michall/roeizucker/phewas_run/runs1/2610:2619
/cs/labs/michall/roeizucker/phewas_run/runs1/2610:2619/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/3100:3109
/cs/labs/michall/roeizucker/phewas_run/runs1/3100:3109/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/480:489
/cs/labs/michall/roeizucker/phewas_run/runs1/480:489/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/1890:1899
/cs/labs/michall/roeizucker/phewas_run/runs1/1890:1899/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4360:4369
/cs/labs/michall/roeizucker/phewas_run/runs1/4360:4369/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/960:969
/cs/labs/michall/roeizucker/phewas_run/runs1/960:969/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4820:4829
/cs/labs/michall/roeizucker/phewas_run/runs1/4820:4829/GWAS/results
cp: No match.
/cs/labs/michall/roeizucker/phewas_run/runs1/600:609
/cs/labs/michall/roeizucker/phewas_run/runs1/600:609/GWAS/results
/cs/labs/micha

/cs/labs/michall/roeizucker/phewas_run/runs1/860:869
/cs/labs/michall/roeizucker/phewas_run/runs1/860:869/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/1120:1129
/cs/labs/michall/roeizucker/phewas_run/runs1/1120:1129/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/3410:3419
/cs/labs/michall/roeizucker/phewas_run/runs1/3410:3419/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/2300:2309
/cs/labs/michall/roeizucker/phewas_run/runs1/2300:2309/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4670:4679
/cs/labs/michall/roeizucker/phewas_run/runs1/4670:4679/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/90:99
/cs/labs/michall/roeizucker/phewas_run/runs1/90:99/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/2840:2849
/cs/labs/michall/roeizucker/phewas_run/runs1/2840:2849/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/1170:1179
/cs/labs/michall/roeizucker/phewas_run/runs1/1170:1179/GWAS/results
/cs/labs/michall/roeizucker/

/cs/labs/michall/roeizucker/phewas_run/runs1/3870:3879
/cs/labs/michall/roeizucker/phewas_run/runs1/3870:3879/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4150:4159
/cs/labs/michall/roeizucker/phewas_run/runs1/4150:4159/GWAS/results
cp: No match.
/cs/labs/michall/roeizucker/phewas_run/runs1/2390:2399
/cs/labs/michall/roeizucker/phewas_run/runs1/2390:2399/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/3330:3339
/cs/labs/michall/roeizucker/phewas_run/runs1/3330:3339/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/3480:3489
/cs/labs/michall/roeizucker/phewas_run/runs1/3480:3489/GWAS/results
cp: No match.
/cs/labs/michall/roeizucker/phewas_run/runs1/2420:2429
/cs/labs/michall/roeizucker/phewas_run/runs1/2420:2429/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/5060:5069
/cs/labs/michall/roeizucker/phewas_run/runs1/5060:5069/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4770:4779
/cs/labs/michall/roeizucker/phewas_run/runs1/4770:4779/G

cp: No match.
/cs/labs/michall/roeizucker/phewas_run/runs1/1990:1999
/cs/labs/michall/roeizucker/phewas_run/runs1/1990:1999/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4260:4269
/cs/labs/michall/roeizucker/phewas_run/runs1/4260:4269/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/3000:3009
/cs/labs/michall/roeizucker/phewas_run/runs1/3000:3009/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/2710:2719
/cs/labs/michall/roeizucker/phewas_run/runs1/2710:2719/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/1560:1569
/cs/labs/michall/roeizucker/phewas_run/runs1/1560:1569/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4920:4929
/cs/labs/michall/roeizucker/phewas_run/runs1/4920:4929/GWAS/results
cp: No match.
/cs/labs/michall/roeizucker/phewas_run/runs1/4990:4999
/cs/labs/michall/roeizucker/phewas_run/runs1/4990:4999/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/1260:1269
/cs/labs/michall/roeizucker/phewas_run/runs1/1260:1269/G

/cs/labs/michall/roeizucker/phewas_run/runs1/3470:3479
/cs/labs/michall/roeizucker/phewas_run/runs1/3470:3479/GWAS/results
cp: No match.
/cs/labs/michall/roeizucker/phewas_run/runs1/140:149
/cs/labs/michall/roeizucker/phewas_run/runs1/140:149/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/5100:5109
/cs/labs/michall/roeizucker/phewas_run/runs1/5100:5109/GWAS/results
cp: No match.
/cs/labs/michall/roeizucker/phewas_run/runs1/4610:4619
/cs/labs/michall/roeizucker/phewas_run/runs1/4610:4619/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/460:469
/cs/labs/michall/roeizucker/phewas_run/runs1/460:469/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/910:919
/cs/labs/michall/roeizucker/phewas_run/runs1/910:919/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4110:4119
/cs/labs/michall/roeizucker/phewas_run/runs1/4110:4119/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/2460:2469
/cs/labs/michall/roeizucker/phewas_run/runs1/2460:2469/GWAS/results


/cs/labs/michall/roeizucker/phewas_run/runs1/2670:2679
/cs/labs/michall/roeizucker/phewas_run/runs1/2670:2679/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4300:4309
/cs/labs/michall/roeizucker/phewas_run/runs1/4300:4309/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/4400:4409
/cs/labs/michall/roeizucker/phewas_run/runs1/4400:4409/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/1840:1849
/cs/labs/michall/roeizucker/phewas_run/runs1/1840:1849/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/3660:3669
/cs/labs/michall/roeizucker/phewas_run/runs1/3660:3669/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/2170:2179
/cs/labs/michall/roeizucker/phewas_run/runs1/2170:2179/GWAS/results
/cs/labs/michall/roeizucker/phewas_run/runs1/gwas
/cs/labs/michall/roeizucker/phewas_run/runs1/gwas/GWAS/results
cp: No match.
/cs/labs/michall/roeizucker/phewas_run/runs1/1300:1309
/cs/labs/michall/roeizucker/phewas_run/runs1/1300:1309/GWAS/results
/cs/labs/mic

In [209]:
def handle_df(df):
#     df['-logP'] = df['Z_STAT']
    df['-logP'] = -np.log10(df['P'])
    df['ind'] = df['POS']
    df = df.set_index(['#CHROM','POS','REF','ALT'])
    return df

def create_combined_gwas_datafrmes(base_dir,phen,chrom,female_p_range=4,male_p_range=4,all_p_range=4,verbose=False):
    try:
#         print(f"{base_dir}/{phen}_chr{chrom}.PHENO1.glm.logistic")
        male_df = pd.read_csv(f"{base_dir}/{phen}_m_chr{chrom}.PHENO1.glm.logistic",sep="\t")
        # male_df['-logP'] = -np.log10(male_df['P'])
        male_df = handle_df(male_df)

        female_df = pd.read_csv(f"{base_dir}/{phen}_f_chr{chrom}.PHENO1.glm.logistic",sep="\t")
        female_df = handle_df(female_df)

        both_df = pd.read_csv(f"{base_dir}/{phen}_chr{chrom}.PHENO1.glm.logistic",sep="\t")
        both_df = handle_df(both_df)
    except pd.errors.EmptyDataError:
        return -1,-1,-1,-1
    except FileNotFoundError:
        return -1,-1,-1,-1 
    comb_df = pd.DataFrame()
    comb_df["male_neg_logP"] = male_df['-logP']
    comb_df["female_neg_logP"] = female_df['-logP']
    comb_df["both_neg_logP"] = both_df['-logP']
    
    comb_df["malez"] = male_df['Z_STAT']
    comb_df["femalez"] = female_df['Z_STAT']
    comb_df["bothz"] = both_df['Z_STAT']
    comb_df["ID"] =female_df["ID"]
    comb_df['ind'] = female_df['ind']
    comb_df = comb_df.dropna()
    comb_df = comb_df.query('both_neg_logP >=3') #removed to bring everything
    return male_df,female_df,both_df,comb_df


In [219]:
GWAS_outputs_path = "/cs/labs/michall/roeizucker/phewas_run/GWAS_results"
num_counter = 0


In [215]:

def create_multiple_GWAS_files(base_dir,phen,verbose=False):
    if os.path.exists(f"{GWAS_outputs_path}/{phen}_gwas_results.csv"):
        return
    if phen in bads:
        return
    dataframes = []
     
    for chrom in range(1,23):
        male_df,female_df,both_df,comb_df = create_combined_gwas_datafrmes(base_dir,phen.replace("_M","").replace("_F",""),chrom,verbose=verbose)

        if type(male_df) is int:
            continue
        dataframes.append(comb_df)
    if len(dataframes) > 0:
        result = pd.concat(dataframes)
        result.to_csv(f"{GWAS_outputs_path}/{phen}_gwas_results.csv")
        if verbose:
            print(f"{GWAS_outputs_path}/{phen}_gwas_results.csv")
#                 create_graphs_for_phenotype(dir_name + "/GWAS/results",spec["name"])

# create_multiple_GWAS_files()

In [58]:
DISCRIPTION_FILE = "/cs/labs/michall/roeizucker/phewas_run/discription9_no_duplicates.csv"
# discription_df = pd.read_csv(DISCRIPTION_FILE)

In [223]:
discription_df = pd.read_csv(DISCRIPTION_FILE)
discription_df= discription_df[discription_df["source"] != "10krun"]
print(len(discription_df))
counter = 0
bads = []
for name in discription_df["name"]:
    for chrom in range(1,23):
        if not os.path.exists(os.path.join(gwas_results_dir,f"{name.lower()}_chr{chrom}.PHENO1.glm.logistic")):
#             print(name,"does not have all chromosomes",chrom)
#             create_multiple_GWAS_files(gwas_results_dir,name.lower(),verbose=True)
            bads.append(name)
            break
    else:
        counter+=1
        try:
            create_multiple_GWAS_files(gwas_results_dir,name.lower(),verbose=True)
        except:
            bads.append(name)
#             break

print(counter)

5042


/cs/usr/roeizucker/my_python/lib/python3.7/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


2160


In [224]:
len(bads)

2931

In [131]:
pd.read_csv("/cs/labs/michall/roeizucker/phewas_run/runs1/GWAS_results/phecode_200.0_chr11.PHENO1.glm.logistic",nrows=15214)

,#CHROM\tPOS\tID\tREF\tALT\tA1\tTEST\tOBS_CT\tOR\tLOG(OR)_SE\tZ_STAT\tP
0,11\t193112\trs7951297\tT\tC\tT\tADD\t147312\t0...
1,11\t193113\trs200059999\tA\tG\tA\tADD\t267492\...
2,11\t193146\trs61736219\tA\tG\tA\tADD\t86724\t0...
3,11\t193724\trs541237660\tA\tG\tA\tADD\t267416\...
4,11\t193797\trs199816295\tC\tG\tC\tADD\t267519\...
...,...
15209,11\t101375091\trs577860483\tG\tT\tG\tADD\t2675...
15210,11\t101375326\trs146776939\tC\tT\tC\tADD\t2673...
15211,11\t101375506\trs541383617\tT\tG\tT\tADD\t2674...
15212,11\t101375527\trs202230574\tT\tC\tT\tADD\t2674...


## IDK, run on the server i guess?

In [ ]:
# i = 0
finished = False
for i in range(len(phenotypes)):
    for j in range(1,23):
#         print(f"/cs/phd/nadavb/third_party/plink2 --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.fam --pheno {GWAS_PATH}/phenotypes/{phenotypes[i].lower()}.txt --covar {GWAS_PATH}/covariates.txt --out {GWAS_PATH}/results/{phenotypes[i]}_chr{j} --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000")
#         input()
        !{PLINK_PATH} --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps2/small_ch{j}.fam --pheno {GWAS_PATH}/phenotypes/{phenotypes[i].lower()}.txt --covar {GWAS_PATH}/covariates.txt --out {GWAS_PATH}/results/{phenotypes[i]}_chr{j} --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 10 --memory 10000
print("done")
finished = True

In [ ]:
!cd {GWAS_PATH}/results/
!ls -la

In [ ]:
!{PLINK_PATH} --bed /cs/labs/michall/roeizucker/plink_results/reduced_snps/reduced_chr8.bed --bim /cs/labs/michall/roeizucker/plink_results/reduced_snps/reduced_chr8.bim --fam /cs/labs/michall/roeizucker/plink_results/reduced_snps/reduced_chr8.fam --pheno /cs/labs/michall/roeizucker/phewas_run/runs1/0:9/plink_run/phenotypes/Phecode_8.0.txt --covar /cs/labs/michall/roeizucker/phewas_run/runs1/0:9/plink_run/covariates.txt --out /cs/labs/michall/roeizucker/phewas_run/runs1/0:9/plink_run/results/Phecode_8.0 --1 --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 1 --memory 10000

In [ ]:
import os
import re
import json
from datetime import datetime
from collections import Counter

import numpy as np
import pandas as pd

from IPython.display import display

# Create the dataset

In [12]:
# with open('/cs/phd/nadavb/pwas_results/cancer_phenotype_columns.json', 'r') as f:
#     cancer_phenotype_cols = json.load(f)
    
with open(COVARIATE_JSON_PATH, 'r') as f:
    covariate_cols = json.load(f)

cancer_dataset = pd.read_csv(PHEN_PATH)
cancer_phenotype_cols = phenotypes
print(cancer_phenotype_cols)
# print(covariate_cols)
# display(cancer_dataset)
# print("done")

NameError: name 'phenotypes' is not defined

In [ ]:
for cancer_phenotype_col in cancer_phenotype_cols:
    print(cancer_phenotype_col)
    cancer_file_name = cancer_phenotype_col.lower().replace(' ', '_').replace('-', '_') + '.txt'
    cancer_values = cancer_dataset[['eid', 'eid', cancer_phenotype_col]].dropna()
    cancer_values.to_csv(os.path.join(f"{GWAS_PATH}/phenotypes", cancer_file_name), \
            header = False, index = False, sep = '\t')

print('Done.')

In [ ]:
covariate_values = cancer_dataset[['eid', 'eid'] + [covariate_col for covariate_col in covariate_cols if \
        covariate_col != 'const']].dropna()
covariate_values.to_csv(f"{GWAS_PATH}/covariates.txt", header = False, index = False, sep = '\t')
print('Done.')

# Test a PLINK command on chrY

In [ ]:
afreqs = pd.read_csv('/cs/labs/michall/roeizucker/plink_results/cancer/results/pan_cancer.afreq', sep = '\t')
display(afreqs)

In [ ]:
id_to_AF = 1 - afreqs.set_index('ID')['ALT_FREQS']
plink_results = pd.read_csv('/cs/labs/michall/roeizucker/plink_results/cancer/results/pan_cancer1.PHENO2.glm.logistic', sep = '\t')
plink_results_AF = plink_results['ID'].map(id_to_AF)
plink_results.insert(plink_results.columns.tolist().index('OR'), 'AF', plink_results_AF)
display(plink_results)

# Running everything on the cluster

In [ ]:
# Original parameters:                          --threads 1 --memory 10000 and -c1 --mem=12g
# First re-run parameters (with RERUN = True):  --threads 8 --memory 30000 and -c8 --mem=36g
# Second re-run parameters (with RERUN = True): --threads 32 --memory 100000 and -c32 --mem=128g

RERUN = False

PLINK_CMD_TEMPLATE = f'{PLINK_PATH} --bed %s --bim %s --fam %s --pheno ' + \
        '%s --covar %s --out %s --1 --glm hide-covar no-x-sex --mac 20 --covar-variance-standardize --freq ' + \
        '--threads 32 --memory 100000'
SLURM_CMD_TEMPLATE = 'sbatch --output=/cs/labs/michall/roeizucker/plink_results/logs/slurm_jobs/%s --mem=128g -c32 --time=14-0 --wrap="%s"'
SH_SCRIPT_FIRST_LINE = '#! /bin/tcsh'

OUTPUT_SUBMISSION_SCRIPT_FILE_PATH = '/cs/labs/michall/roeizucker/plink_results/cancer/submit_jobs.sh'
LOG_FILE_NAME_TEMPLATE = 'cancer_2020_04_20_%s.out'
COVAR_FILE_PATH = '/cs/labs/michall/roeizucker/plink_results/cancer/covariates.txt'
PHENOTYPES_DIR = '/cs/labs/michall/roeizucker/plink_results/cancer/phenotypes'
RESULTS_DIR = '/cs/labs/michall/roeizucker/plink_results/cancer/results'
GENETIC_DATA_DIR = '/cs/icore/idomarg/covarathon/001'

genotyping_spec = pd.read_csv('/cs/phd/nadavb/pwas_results/ukbb_raw_marker_genotyping_spec.csv')

def convert_genetic_file_path(original_path):
    updated_path = os.path.join(GENETIC_DATA_DIR, os.path.basename(original_path))
    assert os.path.exists(updated_path)
    return updated_path

with open(OUTPUT_SUBMISSION_SCRIPT_FILE_PATH, 'w') as f:
    
    f.write(SH_SCRIPT_FIRST_LINE + '\n\n')
    
    for cancer_phenotype_col in cancer_phenotype_cols:

        cancer_simplified_name = cancer_phenotype_col.lower().replace(' ', '_').replace('-', '_')
        cancer_phenotype_file_path = os.path.join(PHENOTYPES_DIR, '%s.txt' % cancer_simplified_name)

        for _, (chr_name, bed_file_path, bim_file_path, fam_file_path) in genotyping_spec[['name', 'bed_file_path', \
                'bim_file_path', 'fam_file_path']].iterrows():
            
            if RERUN:
                if (results_status['is_done'] & (results_status['chrom'] == chr_name) & \
                        (results_status['phenotype_name'] == cancer_simplified_name)).sum() > 0:
                    continue

            bed_file_path, bim_file_path, fam_file_path = map(convert_genetic_file_path, [bed_file_path, bim_file_path, \
                    fam_file_path])
            task_name = 'chr%s_%s' % (chr_name, cancer_simplified_name)
            output_path_prefix = os.path.join(RESULTS_DIR, task_name)
            plink_cmd = PLINK_CMD_TEMPLATE % (bed_file_path, bim_file_path, fam_file_path, cancer_phenotype_file_path, \
                    COVAR_FILE_PATH, output_path_prefix)
            log_file_name = LOG_FILE_NAME_TEMPLATE % task_name
            slurm_cmd = SLURM_CMD_TEMPLATE % (log_file_name, plink_cmd)
            f.write(slurm_cmd + '\n')
            
print('Submission script is ready: %s' % OUTPUT_SUBMISSION_SCRIPT_FILE_PATH)

In [ ]:
!head /cs/labs/michall/roeizucker/plink_results/cancer/submit_jobs.sh

# Validating that it's all finished

In [ ]:
# Viewing the log files to see which jobs have successfully finished.

RESULTS_DIR = '/cs/phd/nadavb/plink_results/cancer/results'

LOG_FILE_NAME_PATTERN = re.compile(r'chr([\dXYMT]+)_(\S+)\.log')

log_file_paths = [os.path.join(RESULTS_DIR, file_name) for file_name in os.listdir(RESULTS_DIR) if \
        file_name.endswith('.log')]

results_status = []

for log_file_path in log_file_paths:
    
    (chrom, phenotype_name), = LOG_FILE_NAME_PATTERN.findall(os.path.basename(log_file_path))
    
    with open(log_file_path, 'r') as f:
        is_done = 'done.' in f.read()
        
    results_status.append((phenotype_name, chrom, is_done, log_file_path))
    
results_status = pd.DataFrame(results_status, columns = ['phenotype_name', 'chrom', 'is_done', 'log_file'])\
        .sort_values(['phenotype_name', 'chrom'])
display(results_status)

In [ ]:
existing_combinatios = [(result['phenotype_name'], result['chrom']) for _, result in results_status.iterrows()]
missing_combinatios = 0

for phenotype_name in sorted(results_status['phenotype_name'].unique()):
    for chrom in sorted(results_status['chrom'].unique()):
        
        combination = (phenotype_name, chrom)
        
        if combination not in existing_combinatios:
            print(combination)
            missing_combinatios += 1
            
print('There are %d missing log files.' % missing_combinatios)

In [ ]:
results_status['is_done'].value_counts()

In [ ]:
results_status.loc[~results_status['is_done']]

In [ ]:
results_status.loc[~results_status['is_done'], 'log_file'].tolist()

# Collecting the results

In [ ]:
EXPECTED_MISSING_RESULTS = {
    # cancer_simplified_name, chr_name
    ('breast_cancer', 'Y'),
    ('epithelial_ovarian_cancer', 'Y'),
}

COMBINED_RESULTS_DIR = '/cs/phd/nadavb/plink_results/cancer/combined_results'

for cancer_phenotype_col in cancer_phenotype_cols:
    
    cancer_simplified_name = cancer_phenotype_col.lower().replace(' ', '_').replace('-', '_')
    cancer_gwas_results = []
    
    for chr_name in genotyping_spec['name']:
        
        if (cancer_simplified_name, chr_name) in EXPECTED_MISSING_RESULTS:
            continue
        
        result_path_prefix = os.path.join(RESULTS_DIR, 'chr%s_%s' % (chr_name, cancer_simplified_name))
        
        chrom_gwas_results = pd.read_csv(result_path_prefix + '.PHENO2.glm.logistic', sep = '\t')\
                .rename(columns = {'#CHROM': 'CHROM'})
        chrom_AF_results = pd.read_csv(result_path_prefix + '.afreq', sep = '\t').rename(columns = {'#CHROM': 'CHROM'})
        snp_id_to_af = 1 - chrom_AF_results.set_index('ID')['ALT_FREQS']
        chrom_gwas_results['AF'] = chrom_gwas_results['ID'].map(snp_id_to_af)
        cancer_gwas_results.append(chrom_gwas_results)
        
    cancer_gwas_results = pd.concat(cancer_gwas_results).reset_index(drop = True)
    print('%s: %d variant results.' % (cancer_simplified_name, len(cancer_gwas_results)))
    cancer_gwas_results.to_csv(os.path.join(COMBINED_RESULTS_DIR, cancer_simplified_name + '.csv'), index = False)

print('Done.')